In [1]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import chi2_contingency 
import scipy.stats as scs

In [2]:
df = pd.read_excel('sexist_survey_num_2.xlsx', sheet_name= 'Sheet1')
df.head()

,Unnamed: 0,Timestamp,gender,career_stage,career_cat,employent_type,background,current_location,discipline,leaky_pipeline,...,dum_current_location_North_America,dum_current_location_Oceania,dum_current_location_South_America,f_interactions_gender_bias,m_interactions_gender_bias,f_gender_biased,m_gender_biased,g_q_academic_positions,g_q_awards,g_q_peer_review
0,0,4.1.2018 10:25:44,1,PhD student,2,1,Europe,Europe,Earth and Space Science Informatics,0,...,0,0,0,4,2,2,2,2,2,2
1,1,3.26.2018 20:01:23,2,"10 years as postdoc and lecturer, now guest le...",9,1,Europe,Europe,Geosciences Instrumentation and Data Systems,1,...,0,0,0,1,1,1,1,3,2,1
2,2,3.25.2018 17:06:22,1,Assistant Professor / lecturer (or equivalent),5,1,Europe,Europe,"Stratigraphy, Sedimentology & Palaeontology",1,...,0,0,0,1,1,3,2,4,4,4
3,3,3.25.2018 17:14:23,1,Assistant Professor / lecturer (or equivalent),5,1,Europe,Europe,"Geochemistry, Mineralogy, Petrology & Volcanology",1,...,0,0,0,4,3,4,1,1,1,1
4,4,3.25.2018 20:30:41,1,Assistant Professor / lecturer (or equivalent),5,2,Europe,Europe,Hydrological Sciences,1,...,0,0,0,1,1,2,4,5,5,1


In [3]:
for column_name in df.columns:
    print(column_name)

Unnamed: 0
Timestamp
gender
career_stage
career_cat
employent_type
background
current_location
discipline
leaky_pipeline
early_career_mumber
tenured_number
gender_imb_exp
gender_distr_imp
gender_bias_institution
gender_bias_org
discuss_bias
training
target_programs
usefulness_rating
importance_role_models
role_models_gender
family
dum_background_Africa
dum_background_Asia
dum_background_Europe
dum_background_North_America
dum_background_Oceania
dum_background_South_America
dum_background_missing
dum_current_location_Africa
dum_current_location_Asia
dum_current_location_Europe
dum_current_location_North_America
dum_current_location_Oceania
dum_current_location_South_America
f_interactions_gender_bias
m_interactions_gender_bias
f_gender_biased
m_gender_biased
g_q_academic_positions
g_q_awards
g_q_peer_review


In [4]:
location_cat = {"current_location": {"Africa": 1, 
                                     "Asia": 2,
                                    "Europe": 3, "North America": 4, 
                                     "Oceania": 5,
                                    "South America": 6
                                    }}

In [5]:
df = df.replace(location_cat)
df.head()

,Unnamed: 0,Timestamp,gender,career_stage,career_cat,employent_type,background,current_location,discipline,leaky_pipeline,...,dum_current_location_North_America,dum_current_location_Oceania,dum_current_location_South_America,f_interactions_gender_bias,m_interactions_gender_bias,f_gender_biased,m_gender_biased,g_q_academic_positions,g_q_awards,g_q_peer_review
0,0,4.1.2018 10:25:44,1,PhD student,2,1,Europe,3,Earth and Space Science Informatics,0,...,0,0,0,4,2,2,2,2,2,2
1,1,3.26.2018 20:01:23,2,"10 years as postdoc and lecturer, now guest le...",9,1,Europe,3,Geosciences Instrumentation and Data Systems,1,...,0,0,0,1,1,1,1,3,2,1
2,2,3.25.2018 17:06:22,1,Assistant Professor / lecturer (or equivalent),5,1,Europe,3,"Stratigraphy, Sedimentology & Palaeontology",1,...,0,0,0,1,1,3,2,4,4,4
3,3,3.25.2018 17:14:23,1,Assistant Professor / lecturer (or equivalent),5,1,Europe,3,"Geochemistry, Mineralogy, Petrology & Volcanology",1,...,0,0,0,4,3,4,1,1,1,1
4,4,3.25.2018 20:30:41,1,Assistant Professor / lecturer (or equivalent),5,2,Europe,3,Hydrological Sciences,1,...,0,0,0,1,1,2,4,5,5,1


In [6]:
print(df['career_cat'].value_counts())
print(df['career_cat'].value_counts(normalize = True))

2    430
3    235
4    120
1    118
6    108
5    106
7     92
8      6
9      5
Name: career_cat, dtype: int64
2    0.352459
3    0.192623
4    0.098361
1    0.096721
6    0.088525
5    0.086885
7    0.075410
8    0.004918
9    0.004098
Name: career_cat, dtype: float64


In [7]:
background_clean = df['dum_background_missing'] != 1
df_clean = df[background_clean]
df_clean.shape

(1216, 43)

In [8]:
print(df['gender'].value_counts())
print(df['gender'].value_counts(normalize = True))

1    818
2    402
Name: gender, dtype: int64
1    0.670492
2    0.329508
Name: gender, dtype: float64


In [9]:
columns_to_merge = ['gender_bias_institution',
                    'gender_bias_org'
                   ]

def merge_cols(row):
    for column in columns_to_merge:
        if row[column] == 3:
            return 1
        else:
            continue
    return 0
    
    
df['gender_biaas_experience'] = df.apply(merge_cols, axis=1)

In [10]:
print(df['gender_biaas_experience'].value_counts())
print(df['gender_biaas_experience'].value_counts(normalize = True))
total_count_woman = 0
total_count_man = 0
count_woman_answered_1 = 0
count_man_answered_1 = 0
    
for index, row in df.iterrows():
    gender = row['gender']
    if gender == 1: #woman
        total_count_woman += 1
        if row['gender_biaas_experience'] == 0:
            count_woman_answered_1 +=1
    
    else: #men
        total_count_man += 1
        
        if row['gender_biaas_experience'] == 0:
            count_man_answered_1 += 1
        
            
print(count_man_answered_1)
pct_man = count_man_answered_1 / total_count_man *100
print(pct_man)
print(count_woman_answered_1)
pct_woman = count_woman_answered_1 / total_count_woman *100
print(pct_woman)

0    904
1    316
Name: gender_biaas_experience, dtype: int64
0    0.740984
1    0.259016
Name: gender_biaas_experience, dtype: float64
353
87.81094527363184
551
67.35941320293398


In [11]:
contigency = pd.crosstab(df['gender_biaas_experience'], df['gender']) 
c, p, dof, expected = chi2_contingency(contigency)
print(c, p, dof )
print(contigency)

57.67943602733424 3.085113508676647e-14 1
gender                     1    2
gender_biaas_experience          
0                        551  353
1                        267   49


In [12]:
print(df['employent_type'].value_counts())
print(df['employent_type'].value_counts(normalize = True))

1    975
2    245
Name: employent_type, dtype: int64
1    0.79918
2    0.20082
Name: employent_type, dtype: float64


In [13]:
selected_columns = [column for column in df_clean.columns if column.startswith('dum_background_')]
selected_columns.remove('dum_background_missing')
for col_name in df_clean:
    if col_name in selected_columns:
        counts = df_clean[col_name].value_counts()
        proportions = df_clean[col_name].value_counts(normalize = True)
        normalized = round(proportions  * 100, 2)
        print(counts)
        print(normalized)


0    1204
1      12
Name: dum_background_Africa, dtype: int64
0    99.01
1     0.99
Name: dum_background_Africa, dtype: float64
0    1137
1      79
Name: dum_background_Asia, dtype: int64
0    93.5
1     6.5
Name: dum_background_Asia, dtype: float64
1    656
0    560
Name: dum_background_Europe, dtype: int64
1    53.95
0    46.05
Name: dum_background_Europe, dtype: float64
0    827
1    389
Name: dum_background_North_America, dtype: int64
0    68.01
1    31.99
Name: dum_background_North_America, dtype: float64
0    1190
1      26
Name: dum_background_Oceania, dtype: int64
0    97.86
1     2.14
Name: dum_background_Oceania, dtype: float64
0    1162
1      54
Name: dum_background_South_America, dtype: int64
0    95.56
1     4.44
Name: dum_background_South_America, dtype: float64


In [14]:
selected_columns = [column for column in df.columns if column.startswith('current_location_')]
for col_name in df:
    if col_name in selected_columns:
        counts = df[col_name].value_counts()
        proportions = df[col_name].value_counts(normalize = True)
        normalized = round(proportions  * 100, 2)
        print(counts)
        print(normalized)

In [15]:
print(df['early_career_mumber'].value_counts())
print(df['early_career_mumber'].value_counts(normalize = True))
print(df['tenured_number'].value_counts())
print(df['tenured_number'].value_counts(normalize = True))

1    508
2    471
3    121
4    120
Name: early_career_mumber, dtype: int64
1    0.416393
2    0.386066
3    0.099180
4    0.098361
Name: early_career_mumber, dtype: float64
1    997
2    112
3     72
4     39
Name: tenured_number, dtype: int64
1    0.817213
2    0.091803
3    0.059016
4    0.031967
Name: tenured_number, dtype: float64


In [16]:
tenured_more_men = list()
for cell in df['tenured_number']:
    if cell == 1:
        tenured_more_men.append(1)
    else:
        tenured_more_men.append(0)
df['tenured_more_men'] = tenured_more_men


early_career_more_men = list()
for cell in df['early_career_mumber']:
    if cell == 1:
        early_career_more_men.append(1)
    else:
        early_career_more_men.append(0)
df['early_career_more_men'] = early_career_more_men


In [17]:
contigency = pd.crosstab(df['tenured_more_men'], df['early_career_more_men']) 
c, p, dof, expected = chi2_contingency(contigency)
print(c, p, dof )
print(contigency)

59.554380160129526 1.1896107472673136e-14 1
early_career_more_men    0    1
tenured_more_men               
0                      182   41
1                      530  467


In [18]:
columns_to_merge = ['gender_bias_institution',
                    'gender_bias_org'
                   ]

def merge_cols_advantage(row):
    for column in columns_to_merge:
        if row[column] == 2:
            return 1
        else:
            continue
    return 0
    
    
df['gender_bias_advantage'] = df.apply(merge_cols_advantage, axis=1)

In [19]:
columns_to_merge = ['gender_imb_exp',
                    'gender_distr_imp'
                   ]

def merge_cols_imbalance(row):
    for column in columns_to_merge:
        #if row[column] == 3:
            #return 1
        if row[column] == 4:
            return 1
        else:
            continue
    return 0
    
    
df['gender_imbalance'] = df.apply(merge_cols_imbalance, axis=1)

In [20]:
contigency = pd.crosstab(df['gender_imbalance'], df['gender']) 
c, p, dof, expected = chi2_contingency(contigency)
print(c, p, dof )
print(contigency)

13.134838486391768 0.0002898552609746465 1
gender              1    2
gender_imbalance          
0                 434  258
1                 384  144


In [21]:
contigency = pd.crosstab(df['gender_bias_advantage'], df['gender']) 
c, p, dof, expected = chi2_contingency(contigency)
print(c, p, dof )
print(contigency)

total_count_woman = 0
total_count_man = 0
count_woman_answered_1 = 0
count_gender_imp_fem = 0
count_gender_imp_masc = 0
count_man_answered_1 = 0
    
for index, row in df.iterrows():
    gender = row['gender']
    if gender == 1: #woman
        total_count_woman += 1
        if row['gender_bias_advantage'] == 1:
            count_woman_answered_1 +=1
    
    else:
        total_count_man += 1
        if row['gender_bias_advantage'] == 1:
            count_man_answered_1 += 1
        
            
print(count_man_answered_1)
pct_man = count_man_answered_1 / total_count_man *100
print(pct_man)
print(count_woman_answered_1)
pct_woman = count_woman_answered_1 / total_count_woman *100
print(pct_woman)


7.484869630563362 0.006221957374688848 1
gender                   1    2
gender_bias_advantage          
0                      765  357
1                       53   45
45
11.194029850746269
53
6.47921760391198


In [22]:
print(df['gender_imbalance'].value_counts())
print(df['gender_imbalance'].value_counts(normalize = True))

0    692
1    528
Name: gender_imbalance, dtype: int64
0    0.567213
1    0.432787
Name: gender_imbalance, dtype: float64


In [23]:
df['gender_reverse'] = df['gender'].replace({1:2, 2:1})

In [24]:
non_academia = df['career_cat'] != 9
df_clean = df[non_academia]

formula = "gender_imbalance ~ C(gender_reverse) +  C(career_cat) + C(current_location)"

result = sm.Logit.from_formula(formula, data=df_clean).fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.663596
         Iterations 5
                            Results: Logit
Model:                Logit              Pseudo R-squared:   0.029     
Dependent Variable:   gender_imbalance   AIC:                1640.5371 
Date:                 2022-06-06 11:57   BIC:                1711.9721 
No. Observations:     1215               Log-Likelihood:     -806.27   
Df Model:             13                 LL-Null:            -830.66   
Df Residuals:         1201               LLR p-value:        4.8131e-06
Converged:            1.0000             Scale:              1.0000    
No. Iterations:       5.0000                                           
-----------------------------------------------------------------------
                          Coef.  Std.Err.    z    P>|z|   [0.025 0.975]
-----------------------------------------------------------------------
Intercept                -0.2535   0.6484 -0.3910 0.6958 -1.524

In [25]:
non_academia = df['career_cat'] != 9
df_clean = df[non_academia]

locaition = ['dum_current_location_Africa', 
             'dum_current_location_Asia',
             'dum_current_location_Europe',
             'dum_current_location_North_America',
             'dum_current_location_Oceania',
             'dum_current_location_South_America']
                  
formula = "gender_biaas_experience ~ C(gender_reverse) + gender_imbalance + C(career_cat)"
for column in locaition:
    formula += " + " + column
print(result.summary2())

                            Results: Logit
Model:                Logit              Pseudo R-squared:   0.029     
Dependent Variable:   gender_imbalance   AIC:                1640.5371 
Date:                 2022-06-06 11:57   BIC:                1711.9721 
No. Observations:     1215               Log-Likelihood:     -806.27   
Df Model:             13                 LL-Null:            -830.66   
Df Residuals:         1201               LLR p-value:        4.8131e-06
Converged:            1.0000             Scale:              1.0000    
No. Iterations:       5.0000                                           
-----------------------------------------------------------------------
                          Coef.  Std.Err.    z    P>|z|   [0.025 0.975]
-----------------------------------------------------------------------
Intercept                -0.2535   0.6484 -0.3910 0.6958 -1.5244 1.0174
C(gender_reverse)[T.2]    0.4674   0.1297  3.6034 0.0003  0.2132 0.7217
C(career_cat)[T.2]   

In [26]:
contigency = pd.crosstab(df['gender_imbalance'], df['gender']) 
c, p, dof, expected = chi2_contingency(contigency)
print(c, p, dof )
print(contigency)

13.134838486391768 0.0002898552609746465 1
gender              1    2
gender_imbalance          
0                 434  258
1                 384  144


In [27]:
print(df['gender_imbalance'].value_counts())
print(df['gender_imbalance'].value_counts(normalize = True))
total_count_woman = 0
total_count_man = 0
count_woman_answered_1 = 0
count_man_answered_1 = 0
    
for index, row in df.iterrows():
    gender = row['gender']
    if gender == 1: #woman
        total_count_woman += 1
        if row['gender_imbalance'] == 0:
            count_woman_answered_1 +=1
    
    else: #men
        total_count_man += 1
        
        if row['gender_imbalance'] == 0:
            count_man_answered_1 += 1
        
            
print(count_man_answered_1)
pct_man = count_man_answered_1 / total_count_man *100
print(pct_man)
print(count_woman_answered_1)
pct_woman = count_woman_answered_1 / total_count_woman *100
print(pct_woman)

0    692
1    528
Name: gender_imbalance, dtype: int64
0    0.567213
1    0.432787
Name: gender_imbalance, dtype: float64
258
64.17910447761194
434
53.0562347188264


In [28]:
column = df['importance_role_models']
list_3_4 = list()

for value in df['importance_role_models']:
    if value == 3 or value == 4:
        list_3_4.append(1)
    else:
        list_3_4.append(0)
    
    
df['gender_model_imp'] = list_3_4

In [29]:
df = pd.get_dummies(df, columns=['importance_role_models'])
df.head()

,Unnamed: 0,Timestamp,gender,career_stage,career_cat,employent_type,background,current_location,discipline,leaky_pipeline,...,early_career_more_men,gender_bias_advantage,gender_imbalance,gender_reverse,gender_model_imp,importance_role_models_1,importance_role_models_2,importance_role_models_3,importance_role_models_4,importance_role_models_5
0,0,4.1.2018 10:25:44,1,PhD student,2,1,Europe,3,Earth and Space Science Informatics,0,...,0,0,0,2,1,0,0,0,1,0
1,1,3.26.2018 20:01:23,2,"10 years as postdoc and lecturer, now guest le...",9,1,Europe,3,Geosciences Instrumentation and Data Systems,1,...,0,0,0,1,0,1,0,0,0,0
2,2,3.25.2018 17:06:22,1,Assistant Professor / lecturer (or equivalent),5,1,Europe,3,"Stratigraphy, Sedimentology & Palaeontology",1,...,0,1,0,2,0,0,1,0,0,0
3,3,3.25.2018 17:14:23,1,Assistant Professor / lecturer (or equivalent),5,1,Europe,3,"Geochemistry, Mineralogy, Petrology & Volcanology",1,...,0,0,1,2,1,0,0,0,1,0
4,4,3.25.2018 20:30:41,1,Assistant Professor / lecturer (or equivalent),5,2,Europe,3,Hydrological Sciences,1,...,0,0,1,2,0,1,0,0,0,0


In [30]:
total_count = sum(df['gender_model_imp'])
total_count_woman = 0
total_count_man = 0
count_woman_answered_1 = 0
count_gender_imp_fem = 0
count_gender_imp_masc = 0
count_man_answered_1 = 0
    
for index, row in df.iterrows():
    gender = row['gender']
    if gender == 1: #woman
        total_count_woman += 1
        if row['gender_model_imp'] == 1:
            count_woman_answered_1 +=1
            if row['role_models_gender'] == 2:
                count_gender_imp_fem += 1
    else:
        total_count_man += 1
        if row['gender_model_imp'] == 1:
            count_man_answered_1 += 1
            if row['role_models_gender'] == 2:
                count_gender_imp_masc += 1
            
print(count_man_answered_1)
pct_man = count_man_answered_1 / total_count_man *100
pct_masc = count_gender_imp_masc / total_count_man *100
print(pct_man)
print(pct_masc)
print(count_gender_imp_masc)
print(count_woman_answered_1)
print(count_gender_imp_fem)
pct_woman = count_woman_answered_1 / total_count_woman *100
pct_fem = count_gender_imp_fem / total_count_woman *100
print(pct_woman)
print(pct_fem)

250
62.189054726368155
5.721393034825871
23
684
285
83.61858190709046
34.84107579462103


In [31]:
contigency = pd.crosstab(df['gender_model_imp'], df['gender']) 
c, p, dof, expected = chi2_contingency(contigency)
print(c, p, dof )
print(contigency)

67.77973401879326 1.8281894254833301e-16 1
gender              1    2
gender_model_imp          
0                 134  152
1                 684  250


In [32]:
same_gender_rep = df['role_models_gender'] == 2
df_same_gender_rep = df[same_gender_rep]
contigency = pd.crosstab(df_same_gender_rep['gender_model_imp'], df_same_gender_rep['gender']) 
c, p, dof, expected = chi2_contingency(contigency)
print(c, round(p, 3), dof )
print(contigency)

11.933035714285714 0.001 1
gender              1   2
gender_model_imp         
0                  15   7
1                 285  23


In [33]:
for cell in df['gender']:
    if cell == 1:
        print(df['gender_model_imp'].value_counts())
print(df['gender_model_imp'].value_counts(normalize = True))

1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0

Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gend

1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0

1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0

Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gend

Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gender_model_imp, dtype: int64
1    934
0    286
Name: gend

In [34]:
only_fem = df['gender'] != 2
df_clean = df[only_fem]
print(df_clean['gender_model_imp'].value_counts())
print(df_clean['gender_model_imp'].value_counts(normalize = True))

1    684
0    134
Name: gender_model_imp, dtype: int64
1    0.836186
0    0.163814
Name: gender_model_imp, dtype: float64


In [35]:
only_masc = df['gender'] != 1
df_clean = df[only_masc]
print(df_clean['gender_model_imp'].value_counts())
print(df_clean['gender_model_imp'].value_counts(normalize = True))

1    250
0    152
Name: gender_model_imp, dtype: int64
1    0.621891
0    0.378109
Name: gender_model_imp, dtype: float64
